In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/tools/tokenization.py

In [ ]:
!pip install sentencepiece

In [ ]:
!pip install tensorflow_hub

In [ ]:
!pip install torch

In [ ]:
import nltk
nltk.download('punkt')
nltk.download("stopwords")
nltk.download("words")
from nltk.corpus import stopwords
from nltk.corpus import words

In [ ]:
import pandas as pd
import numpy as np
from random import seed
from random import randint
import random
import string
import re
import matplotlib.pyplot as plt

In [ ]:
from tensorflow.keras.layers import Dense, Input, Reshape, Conv1D, Conv2D, BatchNormalization, MaxPooling1D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, Sequential, load_model
import tensorflow_hub as hub
import tensorflow as tf
from sklearn.model_selection import train_test_split
import tokenization
from sklearn.model_selection import StratifiedKFold

In [ ]:
gpu_options = tf.compat.v1.GPUOptions(allow_growth=True)
session = tf.compat.v1.InteractiveSession(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

In [ ]:
seed(1)
stop_words = stopwords.words('english')
words=words.words()

In [ ]:
dataset = pd.read_csv('../input/netbeans-preprocessed5/netbeans_preprocessed_os.csv')
dataset

In [ ]:
dataset.dropna(subset = ["description1"], inplace=True)
dataset.dropna(subset = ["description2"], inplace=True)

In [ ]:
# dataset.loc[dataset['bug_id'] == 15957]

In [ ]:
for index, row in dataset.iterrows():
    doc1 = 'Product:' + row['product'] + '.Component:' + row['component'] + '.Description:' + row['description1'] #+ '.Summary:' + row['short_desc1']
#     print(row['bug_id'])
#     print(row['description2'])
    doc2 = 'Product:' + row['product'] + '.Component:' + row['component'] + '.Description:' + row['description2'] #+ '.Summary:' + row['short_desc2']
    dataset.loc[index, 'doc1'] = doc1
    dataset.loc[index, 'doc2'] = doc2

In [ ]:
for index, row in dataset.iterrows():
    desc = row['doc1'] + row['doc2']
    dataset.loc[index, 'description'] = desc

In [ ]:
def reset_column_names():
  dataset.drop('description', axis=1, inplace=True)
#   dataset.drop('short_desc1', axis=1, inplace=True)
#   dataset.drop('description2', axis=1, inplace=True)
#   dataset.drop('short_desc2', axis=1, inplace=True)

  dataset.rename(columns={'description_clean':'description'}, inplace=True) #,'short_desc1_clean':'short_desc1','description2_clean':'description2','short_desc2_clean':'short_desc2'

In [ ]:
def preprocess_text_content(text):
    text = text.lower()
    cleaned_text = re.sub(r" +", " ", text) #remove extra white spaces
    
    return cleaned_text

In [ ]:
dataset['description_clean']= dataset['description'].apply(lambda x:preprocess_text_content(x))
dataset.head()

In [ ]:
reset_column_names()

In [ ]:

### Add tokens to the data make it BERT compatible
def bert_encode(texts, tokenizer, max_len=512):
    print('encode')
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
        text = [w for w in text if w.lower() not in stop_words]
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)

    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [ ]:
def build_model(bert_layer, max_len=512):
    print('build model')
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :] 
   
#     clf_output = Reshape((1024,1))(clf_output)
#     print(clf_output)
    
#     x = Conv1D(8, 3, activation='relu', padding='same')(clf_output)
#     x = BatchNormalization()(x)
#     x = MaxPooling1D()(x)
#     x = Conv1D(32, 3, activation='relu', padding='same')(x)
#     x = BatchNormalization()(x)
#     x = MaxPooling1D()(x)
#     x = Flatten()(x)
#     out = Dense(1, activation='sigmoid')(x)
    out = Dense(1, activation='sigmoid')(clf_output)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=2e-6), loss='binary_crossentropy', metrics=['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
    
    return model

In [ ]:
train, test = train_test_split(dataset, test_size=1)

In [ ]:
test.shape

In [ ]:
FullTokenizer = tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1", trainable=True)

In [ ]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
lower_cased = bert_layer.resolved_object.do_lower_case.numpy()

tokenizer = FullTokenizer(vocab_file, lower_cased)

In [ ]:
model = load_model("../input/openofficebertmodel/openoffice-bert-model")

In [ ]:
test_input = bert_encode(test['description'], tokenizer, max_len=160)
test_labels = test.duplicate.values

In [ ]:
scores = model.evaluate(test_input, test_labels, verbose=0)
f1score = (2 * scores[2]*100 * scores[3]*100)/(scores[2]*100 + scores[3]*100)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
print("%s: %.2f%%" % (model.metrics_names[2], scores[2]*100))
print("%s: %.2f%%" % (model.metrics_names[3], scores[3]*100))
print("f1 score: ", (f1score))